# Optimization Modeling II

**Learning Objective:**

- Articulating the decision, objective and constraints in English. (Model)
- Translating from English description to concrete math formulation. (Model)
- Translating from a concrete math formulation to an abstract math formulation. (Analyze)
- Translating from an abstract math formulation to Python code. (Code)

**Recall Exercise 3 from last class: [See last class's course notes for problem statement, English description and concrete math formulation](21-Optimization Modeling I.ipynb)**


## Abstract Math Formulation

**Input Data:** 

- $I$: the set of sites.
- $J$: the set of districts.
- $a_{ij}$: a binary variable for whether site $i$ is within three minutes of district $j$. (This data can be obtained from the distance table by checking whether each entry is no more than $3$.)

**Decision variables:** For each site $i \in I$, let $x_i$ be a binary variable denoting whether we place an emergency vehicle at this site.

$$\begin{aligned}
\text{Minimize} && \sum_{i \in I} x_i \\
\text{subject to:} \\
\text{(Close by site exists)} && \sum_{i \in I}a_{ij}x_i & \ge 1 && \text{ for each district $j$}. \\
&& x_i & \in \{0,1\} && \text{ for each site $i$.}
\end{aligned}$$

## Python Code

### (For Illustration Only) Writing data to file
In real applications the following code is not needed as the data is already given. This code is provided as an additional illustration of how to write data to a file. See the screenshot of the file we created below.


In [1]:
columns=['District',1, 2, 3, 4, 5, 6, 7]

table=[[1, 5, 3, 4, 3, 8, 9, 0],
[2, 3, 6, 5, 4, 8, 0, 3],
[3, 4, 3, 6, 8, 10, 3, 2],
[4, 6, 0, 2, 7, 3, 2, 5],
[5, 2, 8, 2, 5, 0, 6, 8],
[6, 2, 6, 4, 0, 7, 3, 5],
[7, 0, 12, 5, 5, 5, 7, 2],
[8, 10, 9, 0, 2, 3, 5, 7],
[9, 2, 4, 5, 7, 3, 4, 5]]

import pandas as pd
dist=pd.DataFrame(table,columns=columns)
dist.to_excel('dist.xlsx',index=False)

![excel screenshot](22-dist_excel.png)

### Reading in and preparing input data

We first read in the distance data given by the problem. Note that the index column is set to column 0, in order to obtain the district numbers as the row indices of the DataFrame. 

In [2]:
import pandas as pd
dist=pd.read_excel('dist.xlsx',index_col=0)
dist

,1,2,3,4,5,6,7
District,,,,,,,
1,5,3,4,3,8,9,0
2,3,6,5,4,8,0,3
3,4,3,6,8,10,3,2
4,6,0,2,7,3,2,5
5,2,8,2,5,0,6,8
6,2,6,4,0,7,3,5
7,0,12,5,5,5,7,2
8,10,9,0,2,3,5,7
9,2,4,5,7,3,4,5


Given the above data, the set of sites and set of districts can be directly obtained, without hard coding in anything.

In [3]:
sites=dist.columns
sites

Int64Index([1, 2, 3, 4, 5, 6, 7], dtype='int64')

In [4]:
districts=dist.index
districts

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64', name='District')

The following code transforms the input data into the $a$ matrix that we need for the abstract formulation above.

In [5]:
a=(dist<=3)*1
a

,1,2,3,4,5,6,7
District,,,,,,,
1,0,1,0,1,0,0,1
2,1,0,0,0,0,1,1
3,0,1,0,0,0,1,1
4,0,1,1,0,1,1,0
5,1,0,1,0,1,0,0
6,1,0,0,1,0,1,0
7,1,0,0,0,0,0,1
8,0,0,1,1,1,0,0
9,1,0,0,0,1,0,0


### Gurobi formulation

In [6]:
import gurobipy as grb
mod=grb.Model()
x={}
for i in sites:
    x[i]=mod.addVar(vtype=grb.GRB.BINARY)
mod.setObjective(sum(x[i] for i in sites),sense=grb.GRB.MINIMIZE)
for j in districts:
    mod.addConstr(sum(a.loc[j,i]*x[i] for i in sites)>=1)
    

### Running the code and obtaining the optimal set of sites

In [7]:
mod.optimize()

print('\nMinimum number of sites = ',mod.ObjVal)
selected=[i for i in sites if x[i].x>0]
print('List of sites selected = ',selected)

Optimize a model with 9 rows, 7 columns and 26 nonzeros
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 2 rows and 1 columns
Presolve time: 0.00s
Presolved: 7 rows, 6 columns, 18 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)

Root relaxation: cutoff, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         3.00000    3.00000  0.00%     -    0s

Explored 0 nodes (4 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%

Minim

The statement 

```python
selected=[i for i in sites if x[i].x>0]
```

creates a list named `selected` by list comprehension. (See the [Python reference document from before Exam 1](../Supplementary/Reference for Python Constructs (Exam 1).ipynb).) [See this webpage for additional explanation and practice](http://www.pythonforbeginners.com/basics/list-comprehensions-in-python)

The above code is logically equivalent to the longer code below

```python
selected=[]
for i in sites:
    if x[i].x>0:
        selected.append(i)
```

Note that we use `append` because `selected` is a list object. Moreover, `x[i]` is the Gurobi decision variable that we created with `mod.addVar` for site $i$, and `x[i].x` is the numerical value of this decision variable. [See the last part of the online course notes for 16-LP Duality for all basic Gurobi commands](16-LP Duality.ipynb). 

# Additional Practice Problems

## Exercise 4

Finco Investment Corporation wishes to determine an investment strategy for the firm for the next 3 years. At present (Year 0), $100,000$ is available for investment. The goal is to maximize the cash on hand at the end of Year 3. 

There are five investment options, labeled $A$, $B$, $C$, $D$, and $E$. The cash flow associated with investing \$1 in each investment is given in the table below. 

| Investment option | Now (Year 0) | Year 1 | Year 2 | Year 3 |
|:--:|--|--|--|--|
| A | -\$1.00 | \$0.50 | \$1.00 | 0 |
| B | \$0.00 | -\$1.00 | \$0.50 | \$1.00 |
| C | -\$1.00 | \$1.20 | 0 | 0 |
| D | -\$1.00 | 0 | 0 | \$1.90 |
| E | 0 | 0 | -\$1.00 | \$1.50 |

For example, Option A must be invested this year, and each dollar of cash outflow now returns \$0.50 in Year 1 and one dollar in Year 2. Option E mst be invested in Year 2 and each dollar returns \$1.50 the next year.

To ensure that the company’s portfolio is diversified, Finco required that at most \$75,000 be placed in any single investment option. Returns from investments can be reinvested in the same year. For example, the positive cash flow received from Option C in Year 1 can be reinvested immediately in Option B. However, Finco cannot borrow funds, so net cash on hand must be non-negative in all years. Formulate this as an optimization problem.

## Exercise 5

The following table provides the cost shipping for one-pound packages, from 7 of Amazon’s fulfillment centers (FC) to 4 regions. 

| Region  | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|--|--|--|--|--|--|--|--|
| A. Kings County, NY | 20.25 | 7.70 | 24.59 | 23.26 | 7.69 | 7.70 | 7.69 |
| B. Los Angeles County, CA | 18.43 | 23.30 | 7.69 | 7.69 | 24.16 | 22.12 | 24.91 |
| C. King County, WA | 21.28 | 24.18 | 7.70 | 17.67 | 23.91 | 22.98 | 24.57 |
| D. Harris County, TX | 7.69 | 7.70 | 18.73 | 7.71 | 18.79 | 7.70 | 19.47 |

A shipping cost of $10 or less indicates that the package will be transported via ground shipping; otherwise, it will be transported via air shipping. 

For a certain item that weights a pound, Amazon would like to stock it in as few FCs as possible, while guaranteeing that it can fulfill demand in all 4 regions via ground shipping. Moreover, 

- the item must be stocked in at least one of FCs 5 or 7;
- the item cannot be stocked in FC 4 unless it is also stocked in FC 1; 
- if the item is stocked in FC 2, then it cannot also be stocked in FC 3. 

Formulate an optimization problem to find the minimum number of FCs needed. 

## Exercise 6
Your software company has launched a new Analytics product. As sales manager, you are planning to promote the product by sending salesforce to software conventions running concurrently in Los Angeles, Saint Louis, and Detroit. 

You have 6 representatives available at each of your Little Rock and Urbana Branches. You would like to send at least 2 to the Los Angeles convention, 5 to the Saint Louis convention, and at least 4 to the Detroit convention. 

Roundtrip airfare between the locations are as follows: 

|  | Los Angeles | St. Louis | Detroit |
|--|--|--|--|
| Little Rock | 250 | 150 | 200 |
| Urbana | 300 | 200 | 150 |

Formulate an optimization problem to allocate your sales force so as to minimize total airfare. 


## Exercise 7
SALS Marketing Inc. is developing an advertising campaign for a large consumer goods corporation. SALS has promised a plan that will yield the highest possible “exposure rating,” a measure of the ability to reach the appropriate demographic group and generate demand. The options for advertisements with their respective costs (per unit of advertising) and per-unit exposure ratings are given in the table below. 

| Category | Subcategory | Cost/Unit | Exposure/Unit |
|--|--|--|--|
| Magazines | Literary | \$7,500 | 15000 |
|  | News | \$10,000 | 22500 |
|  | Topical | \$15,000 | 24000 |
| Newspapers | Major Evening | \$3,000 | 75000 |
|  | Major Morning | \$2,000 | 37500 |
| Television | Morning | \$20,000 | 275000 |
|  | Midday | \$10,000 | 180000 |
|  | Evening | \$60,000 | 810000 |
| Radio | Morning | \$15,000 | 180000 |
|  | Midday | \$15,000 | 17000 |
|  | Evening | \$10,000 | 16000 |

Of course, certain restrictions exist for the advertising campaign. The client corporation has budgeted \$800,000 for the campaign, but to restrict overexposure to any particular audience it wants no more than \$300,000 put into any one category (Magazine, Newspaper, etc.). Also, to ensure a broad range of exposure, at least \$100,000 must be spent in each category. 

Formulate this problem as a LP or MIP.

Hospital administrators must schedule nurses so that the hospital’s patients are provided with adequate care. At the same time, in the face of tighter competition in the health care industry, they must pay careful attention to keeping costs down. 

## Exercise 8

From historical records, administrators estimated the minimum number of nurses to have on hand for the various times of the day and days of the week, as shown in the following table. 

| Shift | Time | Minimum number of nurses needed |
|--|--|--|
| 1 | Midnight-4am | 5 |
| 2 | 4am-8am | 12 |
| 3 | 8am-noon | 14 |
| 4 | noon-4pm | 8 |
| 5 | 4pm-8pm | 14 |
| 6 | 8pm-Midnight | 10 |

Nurses work 8 hours a day in two consecutive shifts. As a result, in each shift, there are two types of nurses: those that started in the previous shift (and are now working their second shift), and those that just started in this shift (and will be working in the next shift as well). 

Formulate an optimization problem that determines the minimum total number of nurses required to fulfill the schedule above. 

## Exercise 9

Paris has come to you because she needs help paying off her credit card bills. She owes the amounts on her credit cards listed in the following table. 

| Credit Card | Balance | Monthly Rate |
|--|--|--|
| Saks Fifth Avenue | \$20,000 | 0.5% |
| Bloomingdale's | \$50,000 | 1.0% |
| Macy's | \$40,000 | 1.5% |

Paris has agreed not to shop at any of these stores anymore, and she is willing to allocate up to $5,000 per month to pay off these credit cards. All cards must be paid off within 36 months. Paris’ goal is to minimize the total of all her payments.

To solve this problem, you must understand how interests on credit card loans are calculated. To illustrate, suppose Paris pays $5,000 on Saks during month 1. Then her Saks balance at the beginning of month 2 is 20,000 – 5,000 + 0.005*20,000. This is because Paris incurs a 0.5% interest on her balance of 20,000 during month 1. The payment made during month 1 does not affect this interest. 

Help Paris solve her problem by formulating it into a linear program. 
